In [1]:
import numpy as np
import pandas as pd
import altair as alt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import duplicate as dp
from functools import partial

# https://altair-viz.github.io/user_guide/display_frontends.html#displaying-in-the-jupyter-notebook
alt.renderers.enable('mimetype')

RendererRegistry.enable('mimetype')

In [2]:
X, y = make_regression(n_features=2, noise=1.0, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

feature_names = [f'X_{i}' for i in range(X.shape[1])]
data = pd.DataFrame(data=X, columns=feature_names)
data['y'] = y
data.head()

,X_0,X_1,y
0,1.054452,-1.070753,-72.503056
1,-0.362741,-0.634322,-71.853065
2,-0.854096,0.313068,5.064775
3,1.326386,0.298238,66.931834
4,-0.461585,-1.315907,-140.719410


In [3]:
chart = alt.Chart(data).mark_point().encode(
    x='X_0',
    y='y',
    color='X_1:Q'
)
chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [4]:
correlations = data.corr().stack().reset_index().rename(columns={0: 'correlation', 'level_0': 'variable 0', 'level_1': 'variable 1'})
print(correlations.head())
alt.Chart(correlations).mark_rect().encode(
    x='variable 0:N',
    y='variable 1:N',
    color='correlation:Q'
)

  variable 0 variable 1  correlation
0        X_0        X_0     1.000000
1        X_0        X_1    -0.024005
2        X_0          y     0.266984
3        X_1        X_0    -0.024005
4        X_1        X_1     1.000000


<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [8]:
n_duplicates = 4
# print([x.shape for x in [X_train, y_train]])

X_train_dup, y_train_dup, weights_dup = dp.simple_duplicate(X_train, y_train, n_duplicates=n_duplicates)
# print([x.shape for x in [X_train_dup, y_train_dup, weights_dup]])


compare_sk_with_data = partial(
    dp.compare_sk, 
    X_train=X_train, 
    y_train=y_train, 
    X_train_dup=X_train_dup, 
    y_train_dup=y_train_dup, 
    weights_dup=weights_dup,
    X_test=X_test,
    y_test=y_test

)

score_linear, score_linear_dup = compare_sk_with_data(model=LinearRegression)
score_boost, score_boost_dup = compare_sk_with_data(model=GradientBoostingRegressor)

print(score_linear, score_linear_dup)
print(score_boost, score_boost_dup)

g (300, 2) (300, 1)
0.9999288992109249 0.9999288992109249
0.9575872169913093 0.9575162685291969


/Users/jack/opt/anaconda3/envs/further-education/lib/python3.9/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
